In [ ]:
# 读取模拟数据
import pandas as pd
df = pd.read_csv('data_simulation/data_generated/data_simulate_08.csv')
date_columns = ['日期', '上市日期', '公告日期', '计划披露日', '计划开始日', '计划结束日', '离任日期']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
df

# Law Article 4

## cu_4_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | 计划通过本所集中竞价或大宗交易减持股份 |
| constrain | 应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划 |
| contextual_info | nan |
| note | 不考虑是否及时通知公司 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2031 |
| completion_tokens | 7229 |


### 代码实现

In [ ]:
import pandas as pd
import numpy as np

def check_cu_4_1(df):
    '''
    检查cu_4_1合规性：
    主体：上市公司大股东 | 董监高
    条件：存在竞价/大宗交易减持计划
    约束：首次卖出前15交易日披露
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_4_1_subject'] = False
    df['cu_4_1_condition'] = False
    df['cu_4_1_constraint'] = False

    # 1. 验证责任主体
    major_condition = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    director_condition = (df['股东身份'] == '董监高')
    df['cu_4_1_subject'] = major_condition | director_condition

    # 2. 验证触发条件
    df['cu_4_1_condition'] = (
        df['减持方式'].isin(['竞价交易', '大宗交易']) &
        df['存在减持计划']
    )

    # 3. 约束条件处理逻辑
    def process_plan(group):
        group = group.sort_values('日期')
        plans = []
        current_plan = None
        
        for _, row in group.iterrows():
            if row['存在减持计划']:
                if not current_plan:
                    current_plan = {
                        '披露日': row['计划披露日'],
                        '减持记录': [],
                        'start_date': row['日期']
                    }
                if row['当日减持比例'] > 0:
                    current_plan['减持记录'].append(row['日期'])
            else:
                if current_plan:
                    plans.append(current_plan)
                    current_plan = None
        if current_plan:
            plans.append(current_plan)
        
        plan_data = []
        for p in plans:
            first_sell = min(p['减持记录']) if p['减持记录'] else pd.NaT
            plan_data.append({
                '公司简称': group['公司简称'].iloc[0],
                '股东': group['股东'].iloc[0],
                '计划披露日': p['披露日'],
                '首次卖出日': first_sell
            })
        return pd.DataFrame(plan_data)

    # 处理分组并合并数据
    if not df.empty:
        plan_df = df.groupby(['公司简称', '股东'], group_keys=False).apply(process_plan)
        if not plan_df.empty:
            df = df.merge(
                plan_df,
                on=['公司简称', '股东', '计划披露日'],
                how='left'
            )
        else:
            df['首次卖出日'] = pd.NaT

        # 计算交易日序号映射
        trading_dates = pd.Series(df['日期'].unique()).sort_values()
        date_index_map = {date: idx for idx, date in enumerate(trading_dates)}

        # 计算交易日间隔
        df['披露日序号'] = df['计划披露日'].map(date_index_map)
        df['卖出日序号'] = df['首次卖出日'].map(date_index_map)
        df['cu_4_1_constraint'] = (df['卖出日序号'] - df['披露日序号']) >= 15

        # 清理中间列
        df.drop(['披露日序号', '卖出日序号', '首次卖出日'], axis=1, errors='ignore', inplace=True)
    
    return df

In [ ]:
df = check_cu_4_1(df)
df

---

## cu_4_7


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | 披露减持计划 |
| constrain | 每次披露的减持计划中减持时间区间不得超过3个月 |
| contextual_info | nan |
| note | 三个月按照90个自然日计算; 条件"披露减持计划"理解为考察所有存在减持计划的时期.  |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2031 |
| completion_tokens | 5673 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_4_7(df):
    '''
    检查cu_4_7的合规性：
    主体：上市公司大股东 | 董监高
    条件：存在减持计划
    约束：每次披露的减持计划时间区间不超过90个自然日
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_4_7_subject'] = False
    df['cu_4_7_condition'] = df['存在减持计划'].astype(bool)
    df['cu_4_7_constraint'] = None
    
    # 1. 验证责任主体
    major_shareholder_mask = df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东'])
    director_mask = df['股东身份'] == '董监高'
    share_ratio_mask = df['持股比例'] >= 0.05
    df['cu_4_7_subject'] = major_shareholder_mask | director_mask | share_ratio_mask
    
    # 3. 验证约束条件
    def validate_constraints(group):
        if group['存在减持计划'].any():
            # 按计划披露日分组处理多计划情形
            plan_groups = group[group['存在减持计划']].groupby('计划披露日')
            for plan_date, plan_group in plan_groups:
                start_date = plan_group['计划开始日'].iloc[0]
                end_date = plan_group['计划结束日'].iloc[0]
                duration = (end_date - start_date).days + 1  # 包含起始日
                group.loc[plan_group.index, 'cu_4_7_constraint'] = duration <= 90
        return group
    
    # 按公司和股东维度处理所有减持计划
    df = df.groupby(['公司简称', '股东'], group_keys=False).apply(validate_constraints)
    
    return df

In [ ]:
df = check_cu_4_7(df)
df

---

## cu_4_8


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | 拟在3个月内通过集中竞价交易减持股份的总数超过公司股份总数1% |
| constrain | 应当在首次卖出的30个交易日前预先披露减持计划 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2023 |
| completion_tokens | 6579 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_4_8(df):
    '''
    检查cu_4_8的合规性：
    主体：上市公司大股东 | 董监高
    条件：3个月内通过集中竞价减持超1%
    约束：首次卖出前30个交易日披露
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_4_8_subject'] = False
    df['cu_4_8_condition'] = False
    df['cu_4_8_constraint'] = False

    # 1. 验证责任主体
    is_major = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'] == '董监高'
    df.loc[is_major | is_director, 'cu_4_8_subject'] = True

    # 2. 验证触发条件
    valid_condition = (
        (df['减持方式'] == '竞价交易') &
        df['存在减持计划'] &
        (df['计划减持比例'] > 0.01) &
        ((df['计划结束日'] - df['计划开始日']).dt.days <= 90)
    )
    df.loc[valid_condition, 'cu_4_8_condition'] = True

    # 3. 约束条件处理逻辑
    def get_first_sale(group):
        # 按计划披露日分组处理每个减持计划
        plans = []
        for plan_date, plan_group in group.groupby('计划披露日'):
            # 筛选计划期间内的减持记录
            plan_period = plan_group[
                (plan_group['日期'] >= plan_group['计划开始日']) &
                (plan_group['日期'] <= plan_group['计划结束日'])
            ]
            # 找到首次减持日期
            first_sale = plan_period[plan_period['当日减持比例'] > 0]
            first_date = first_sale['日期'].min() if not first_sale.empty else pd.NaT
            plans.append({
                '公司简称': group.name[0],
                '股东': group.name[1],
                '计划披露日': plan_date,
                '首次卖出日': first_date
            })
        return pd.DataFrame(plans) if plans else pd.DataFrame()

    # 获取所有减持计划的首次卖出日
    plan_info = df.groupby(['公司简称', '股东']).apply(get_first_sale).reset_index(drop=True)
    
    # 合并首次卖出日信息
    if not plan_info.empty:
        df = df.merge(
            plan_info,
            on=['公司简称', '股东', '计划披露日'],
            how='left'
        )
    else:
        df['首次卖出日'] = pd.NaT

    # 计算交易日差
    trading_dates = pd.Series(df['日期'].unique()).sort_values().reset_index(drop=True)
    date_map = {date: idx for idx, date in enumerate(trading_dates)}
    
    df['计划披露日_index'] = df['计划披露日'].map(date_map)
    df['首次卖出日_index'] = df['首次卖出日'].map(date_map)
    df['cu_4_8_constraint'] = (df['首次卖出日_index'] - df['计划披露日_index']) >= 30

    # 清理中间列
    df.drop(['计划披露日_index', '首次卖出日_index', '首次卖出日'], 
            axis=1, inplace=True, errors='ignore')

    return df

In [ ]:
df = check_cu_4_8(df)
df

---

# Law Article 9

## cu_9_1


| 字段 | 内容 |
|------|------|
| subject | 控股股东 | 实际控制人 | 董监高 |
| condition | 公司上市时未盈利且处于实现盈利前阶段 |
| constrain | 自公司股票上市之日起2个完整会计年度内不得减持公开发行并上市前股份 |
| contextual_info | nan |
| note | 假设"当日减持比例"记录的都是减持"公开发行并上市前获取的股份"; 对condition的进一步解释: 上市时未盈利, 且上市至今未盈利. |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2065 |
| completion_tokens | 12643 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_9_1(df):
    '''
    检查cu_9_1的合规性：
    主体：控股股东、实际控制人、董监高
    条件：公司上市时未盈利且上市至今未盈利
    约束：上市后两个完整会计年度内不得减持
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_9_1_subject'] = False
    df['cu_9_1_condition'] = False
    df['cu_9_1_constraint'] = True
    
    # 1. 验证责任主体
    df['cu_9_1_subject'] = df['股东身份'].isin(['控股股东', '实际控制人', '董监高'])
    
    # 2. 验证触发条件
    # 计算会计年度并获取年度最大净利润
    df['会计年度'] = df['日期'].dt.year
    annual_profit = df.groupby(['公司简称', '会计年度'])['净利润'].max().reset_index()
    
    # 获取上市年份和上市年净利润
    df['上市年份'] = df['上市日期'].dt.year
    df = df.merge(
        annual_profit,
        left_on=['公司简称', '上市年份'],
        right_on=['公司简称', '会计年度'],
        how='left',
        suffixes=('', '_上市年')
    )
    df.rename(columns={'净利润': '上市年净利润'}, inplace=True)
    
    # 检查所有年度净利润是否非正
    def check_profit(row):
        company = row['公司简称']
        y0 = row['上市年份']
        y_current = row['会计年度_x']
        mask = (annual_profit['公司简称'] == company) & \
               (annual_profit['会计年度'] >= y0) & \
               (annual_profit['会计年度'] <= y_current)
        return (annual_profit.loc[mask, '净利润'] <= 0).all()
    
    df['cu_9_1_condition'] = (df['上市年净利润'] <= 0) & df.apply(check_profit, axis=1)
    
    # 3. 验证约束条件
    df['end_date'] = df['上市年份'].apply(lambda y: pd.Timestamp(year=y+1, month=12, day=31))
    df['cu_9_1_constraint'] = ~((df['日期'] <= df['end_date']) & (df['当日减持比例'] > 0))
    
    # 清理临时列
    df.drop(columns=['会计年度_x', '会计年度_y', '上市年份', '上市年净利润', 'end_date'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_cu_9_1(df)
df

---

## cu_9_3


| 字段 | 内容 |
|------|------|
| subject | 董监高 |
| condition | 在前款规定的2个完整会计年度期间内离职 |
| constrain | 应当继续遵守前款规定的减持限制 |
| contextual_info | nan |
| note | 只需要检查subject和condition, 而constraint项留为None |
| relation | refer_to |
| target | MEU_9_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2065 |
| completion_tokens | 9410 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_9_3(df):
    '''
    检查cu_9_3的合规性：
    主体：董监高
    条件：在上市后前两个完整会计年度期间离职
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_9_3_subject'] = False
    df['cu_9_3_condition'] = False
    df['cu_9_3_constraint'] = None  # 根据要求留空

    # 1. 验证责任主体：当前身份为董监高
    df['cu_9_3_subject'] = df['股东身份'] == '董监高'

    # 2. 验证触发条件：在上市后前两个会计年度期间离职
    def check_condition(row):
        # 仅处理有效主体
        if not row['cu_9_3_subject']:
            return False
        
        # 获取关键日期
        listing_date = row['上市日期']
        resign_date = row['离任日期']
        
        # 缺失日期直接返回False
        if pd.isna(listing_date) or pd.isna(resign_date):
            return False
        
        # 计算会计年度区间（自然年）
        first_year_start = pd.Timestamp(year=listing_date.year, month=1, day=1)
        second_year_end = pd.Timestamp(year=listing_date.year + 1, month=12, day=31)
        
        return first_year_start <= resign_date <= second_year_end

    # 应用条件检查
    df['cu_9_3_condition'] = df.apply(check_condition, axis=1)

    return df

In [ ]:
df = check_cu_9_3(df)
df

---

# Law Article 10

## cu_10_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 |
| condition | 因涉嫌与本上市公司有关的证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2017 |
| completion_tokens | 10060 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_10_1(df):
    '''
    检查cu_10_1的合规性：
    主体：上市公司大股东
    条件：因涉嫌与本公司有关的证券期货违法犯罪被立案调查或侦查期间
    约束：不得减持股份
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_10_1_subject'] = False
    df['cu_10_1_condition'] = False
    df['cu_10_1_constraint'] = True  # 默认未违反约束
    
    # 1. 验证责任主体：上市公司大股东
    is_major = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    df.loc[is_major, 'cu_10_1_subject'] = True
    
    # 2. 验证触发条件：被立案调查或侦查期间
    # 构建股东事件时间线
    periods_dict = {}
    relevant_events = df[['股东', '日期', '股东涉嫌证券期货违法犯罪事件']].dropna(subset=['股东涉嫌证券期货违法犯罪事件'])
    relevant_events = relevant_events[relevant_events['股东涉嫌证券期货违法犯罪事件'].isin([
        '被中国证监会及其派出机构立案调查',
        '中国证监会及其派出机构立案调查结束',
        '被司法机关立案侦查',
        '司法机关立案侦查结束'
    ])]
    
    for shareholder, group in relevant_events.groupby('股东'):
        events = group.sort_values('日期')
        periods = []
        inv_start, inv_end = None, None
        inv_flag = False
        scr_start, scr_end = None, None
        scr_flag = False
        
        for _, row in events.itertuples():
            event = row.股东涉嫌证券期货违法犯罪事件
            date = row.日期
            
            if event == '被中国证监会及其派出机构立案调查':
                inv_start = date
                inv_flag = True
            elif event == '中国证监会及其派出机构立案调查结束' and inv_flag:
                inv_end = date
                periods.append(('investigation', inv_start, inv_end))
                inv_flag = False
            elif event == '被司法机关立案侦查':
                scr_start = date
                scr_flag = True
            elif event == '司法机关立案侦查结束' and scr_flag:
                scr_end = date
                periods.append(('inquiry', scr_start, scr_end))
                scr_flag = False
        
        # 处理未结束的事件
        if inv_flag:
            periods.append(('investigation', inv_start, pd.Timestamp.max))
        if scr_flag:
            periods.append(('inquiry', scr_start, pd.Timestamp.max))
        
        periods_dict[shareholder] = periods
    
    # 判断日期是否在调查/侦查期间
    def is_in_period(row):
        shareholder = row['股东']
        date = row['日期']
        for period_type, start, end in periods_dict.get(shareholder, []):
            if start <= date <= end:
                return True
        return False
    
    df['cu_10_1_condition'] = df.apply(is_in_period, axis=1)
    
    # 3. 验证约束条件：不得减持
    mask = df['cu_10_1_subject'] & df['cu_10_1_condition']
    df.loc[mask, 'cu_10_1_constraint'] = (df.loc[mask, '当日减持比例'] <= 0)
    
    return df

In [ ]:
df = check_cu_10_1(df)
df

---

## cu_10_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 |
| condition | 因涉嫌与本上市公司有关的证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | 6个月按180个自然日计算, 含处罚日当日 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2042 |
| completion_tokens | 6273 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_10_2(df):
    '''
    检查cu_10_2合规性：
    主体：上市公司大股东（控股股东/实际控制人/持股5%以上股东或持股≥5%）
    条件：因涉本公司证券期货犯罪被处罚后未满180天
    约束：不得减持股份
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_10_2_subject'] = False
    df['cu_10_2_condition'] = False
    df['cu_10_2_constraint'] = (df['当日减持比例'] <= 0)  # 独立检查约束条件
    
    # 1. 验证责任主体
    major_shareholder_mask = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    df.loc[major_shareholder_mask, 'cu_10_2_subject'] = True
    
    # 2. 构建处罚日期索引
    penalty_events = df[df['股东涉嫌证券期货违法犯罪事件'].isin(['行政处罚决定作出', '刑事判决作出'])]
    if not penalty_events.empty:
        penalty_dates = penalty_events.groupby(['公司简称', '股东'])['日期'].apply(list)
        
        # 3. 检查处罚后180天窗口期
        def check_penalty_window(row):
            dates = penalty_dates.get((row['公司简称'], row['股东']), [])
            return any((row['日期'] - pd.Timestamp(date)).days <= 180 
                      and (row['日期'] >= pd.Timestamp(date)) for date in dates)
        
        df['cu_10_2_condition'] = df.apply(check_penalty_window, axis=1)
    
    return df

In [ ]:
df = check_cu_10_2(df)
df

---

## cu_10_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 |
| condition | 因涉及与本上市公司有关的违法违规，被本所公开谴责未满3个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | 三个月按照90个自然日计算, 含处罚日当日;  |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2024 |
| completion_tokens | 8941 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_10_3(df):
    '''
    检查cu_10_3的合规性：
    主体：上市公司大股东（控股股东/实际控制人/持股5%以上股东）
    条件：被本所公开谴责未满90自然日
    约束：不得减持股份
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_10_3_subject'] = False
    df['cu_10_3_condition'] = False
    df['cu_10_3_constraint'] = None

    # 1. 验证责任主体（复合逻辑）
    subject_mask = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |  # 身份标签判断
        (df['持股比例'] >= 0.05)  # 持股比例判断
    )
    df.loc[subject_mask, 'cu_10_3_subject'] = True

    # 2. 验证触发条件（需分组处理）
    # 获取所有被公开谴责的记录
    condemn_events = df[df['股东涉嫌证券期货违法犯罪事件'] == '被本所公开谴责']
    # 按股东分组记录所有谴责日期
    condemn_dates = condemn_events.groupby(['公司简称', '股东'])['日期'].apply(list).reset_index()
    # 合并到原数据
    df = df.merge(condemn_dates, on=['公司简称', '股东'], how='left', suffixes=('', '_condemn'))
    
    # 计算自然日差并标记条件
    def check_condemn(row):
        if not isinstance(row['日期_condemn'], list):
            return False
        current_date = row['日期']
        for d in row['日期_condemn']:
            delta = (current_date - d).days
            if 0 <= delta < 90:  # 含处罚当日（delta=0）
                return True
        return False
    
    df['cu_10_3_condition'] = df.apply(check_condemn, axis=1)
    df = df.drop(columns=['日期_condemn'])  # 清理临时列

    # 3. 验证约束条件（独立标记）
    df['cu_10_3_constraint'] = (df['当日减持比例'] <= 0)  # 未减持则合规

    return df

In [ ]:
df = check_cu_10_3(df)
df

---

# Law Article 11

## cu_11_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 上市公司因涉嫌证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2020 |
| completion_tokens | 7194 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_11_1(df):
    '''
    检查cu_11_1的合规性：
    主体：上市公司控股股东 | 实际控制人
    条件：公司因涉嫌证券期货违法犯罪被立案调查或侦查
    约束：不得减持股份
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_11_1_subject'] = False
    df['cu_11_1_condition'] = False
    df['cu_11_1_constraint'] = False

    # 1. 验证责任主体
    valid_subject = df['股东身份'].isin(['控股股东', '实际控制人'])
    df.loc[valid_subject, 'cu_11_1_subject'] = True

    # 2. 验证触发条件
    valid_condition = df['公司涉嫌证券期货违法犯罪事件'].isin([
        '被中国证监会及其派出机构立案调查',
        '被司法机关立案侦查'
    ])
    df.loc[valid_condition, 'cu_11_1_condition'] = True

    # 3. 验证约束条件（独立检查）
    df['cu_11_1_constraint'] = (df['当日减持比例'] <= 0)

    return df

In [ ]:
df = check_cu_11_1(df)
df

---

## cu_11_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 上市公司因涉嫌证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月的 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | 6个月按180个自然日计算 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2039 |
| completion_tokens | 3472 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_11_2(df):
    '''
    检查cu_11_2合规性：
    主体：上市公司控股股东 | 实际控制人
    条件：公司被立案且处罚后未满180天
    约束：期间不得减持股份
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_11_2_subject'] = False
    df['cu_11_2_condition'] = False
    df['cu_11_2_constraint'] = None
    
    # 1. 验证责任主体
    subject_mask = df['股东身份'].isin(['控股股东', '实际控制人'])
    df.loc[subject_mask, 'cu_11_2_subject'] = True
    
    # 2. 处理触发条件
    # 获取公司最新处罚日期（行政处罚决定/刑事判决）
    penalty_events = ['行政处罚决定作出', '刑事判决作出']
    company_penalties = df[df['公司涉嫌证券期货违法犯罪事件'].isin(penalty_events)]
    latest_penalty = company_penalties.groupby('公司简称', group_keys=False)['日期'].max().reset_index(name='latest_penalty_date')
    
    # 合并处罚信息
    df = df.merge(latest_penalty, on='公司简称', how='left')
    
    # 计算自然日间隔
    df['days_since_penalty'] = (df['日期'] - df['latest_penalty_date']).dt.days
    
    # 条件判断：在处罚后180天内且处罚日期有效
    condition_mask = (
        (df['days_since_penalty'] >= 0) &
        (df['days_since_penalty'] <= 180) &
        df['latest_penalty_date'].notna()
    )
    df.loc[condition_mask, 'cu_11_2_condition'] = True
    
    # 3. 验证约束条件
    combined_mask = df['cu_11_2_subject'] & df['cu_11_2_condition']
    # 当日有减持则违规
    df.loc[combined_mask, 'cu_11_2_constraint'] = (df.loc[combined_mask, '当日减持比例'] <= 0)
    
    # 清理中间列
    df.drop(columns=['latest_penalty_date', 'days_since_penalty'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_cu_11_2(df)
df

---

## cu_11_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 上市公司被本所公开谴责未满3个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | 三个月按照90个自然日计算, 含处罚日当日;  |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2023 |
| completion_tokens | 5516 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_11_3(df):
    '''
    检查cu_11_3的合规性：
    主体：上市公司控股股东 | 实际控制人
    条件：上市公司被本所公开谴责未满90天
    约束：不得减持股份
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_11_3_subject'] = False
    df['cu_11_3_condition'] = False
    df['cu_11_3_constraint'] = True  # 默认未减持

    # 1. 验证责任主体
    subject_mask = df['股东身份'].isin(['控股股东', '实际控制人'])
    df.loc[subject_mask, 'cu_11_3_subject'] = True

    # 2. 验证触发条件
    # 获取各公司最新公开谴责日期
    condemn_dates = (df[df['公司涉嫌证券期货违法犯罪事件'] == '被本所公开谴责']
                     .groupby('公司简称')['日期']
                     .max()
                     .reset_index()
                     .rename(columns={'日期': '最新谴责日'}))
    
    # 合并谴责日期到主表
    df = df.merge(condemn_dates, on='公司简称', how='left')
    
    # 计算自然日间隔（含处罚当日）
    if not condemn_dates.empty:
        df['自然日差'] = (df['日期'] - df['最新谴责日']).dt.days
        condition_mask = (df['自然日差'] >= 0) & (df['自然日差'] < 90)
        df.loc[condition_mask, 'cu_11_3_condition'] = True
        df['cu_11_3_condition'] = df['cu_11_3_condition'].fillna(False)
        df.drop(['最新谴责日', '自然日差'], axis=1, inplace=True, errors='ignore')
    else:
        df['cu_11_3_condition'] = False

    # 3. 验证约束条件
    df['cu_11_3_constraint'] = (df['当日减持比例'] == 0)

    return df

In [ ]:
df = check_cu_11_3(df)
df

---

## cu_11_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 |
| condition | 市公司股票因可能触及重大违法强制退市情形，而被本所实施退市风险警示，在本所规定的限制减持期限内的 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2028 |
| completion_tokens | 10501 |


### 代码实现

In [ ]:
import pandas as pd
from collections import defaultdict

def check_cu_11_4(df):
    '''
    检查cu_11_4的合规性：
    主体：上市公司控股股东 | 实际控制人
    条件：公司股票因重大违法被实施退市风险警示且在限制期内
    约束：不得减持股份
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_11_4_subject'] = False
    df['cu_11_4_condition'] = False
    df['cu_11_4_constraint'] = None
    
    # 1. 验证责任主体
    df['cu_11_4_subject'] = df['股东身份'].isin(['控股股东', '实际控制人'])
    
    # 2. 验证触发条件（基于立案调查后180天限制期）
    # 获取公司被立案调查的日期
    investigation_mask = df['公司涉嫌证券期货违法犯罪事件'] == '被中国证监会及其派出机构立案调查'
    investigation_dates = df[investigation_mask][['公司简称', '日期']].drop_duplicates()
    
    # 构建公司限制期区间
    company_periods = defaultdict(list)
    for _, row in investigation_dates.iterrows():
        company = row['公司简称']
        start_date = row['日期']
        end_date = start_date + pd.Timedelta(days=180)
        company_periods[company].append((start_date, end_date))
    
    # 检查日期是否在限制期内
    def is_in_restriction(row):
        company = row['公司简称']
        current_date = row['日期']
        periods = company_periods.get(company, [])
        for start, end in periods:
            if start <= current_date <= end:
                return True
        return False
    
    df['cu_11_4_condition'] = df.apply(is_in_restriction, axis=1)
    
    # 3. 验证约束条件（仅对符合主体和条件的行进行检查）
    mask = df['cu_11_4_subject'] & df['cu_11_4_condition']
    df.loc[mask, 'cu_11_4_constraint'] = (df.loc[mask, '当日减持比例'] <= 0)
    
    return df

In [ ]:
df = check_cu_11_4(df)
df

---

# Law Article 12

## cu_12_1


| 字段 | 内容 |
|------|------|
| subject | 公开发行股票并上市时的控股股东 | 公开发行股票并上市时的实际控制人 |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constrain | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 |
| contextual_info | 股票收盘价以公开发行股票并上市之日为基准向后复权计算 |
| note | 不考虑已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况; 公司上市前30个交易日不会披露减持计划 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2106 |
| completion_tokens | 10107 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_12_1(df):
    '''
    检查cu_12_1的合规性：
    主体：公开发行股票并上市时的控股股东或实际控制人
    条件：首次通过竞价/大宗交易减持计划
    约束：最近20个交易日内无收盘价低于发行价（复权后）
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_12_1_subject'] = False
    df['cu_12_1_condition'] = False
    df['cu_12_1_constraint'] = None
    
    # 1. 验证责任主体：上市时的控股股东或实际控制人
    # 找到每个公司的第一个非空收盘价的日期（上市日期）
    first_close_dates = df[df['收盘价'].notna()].groupby('公司简称')['日期'].min().reset_index()
    first_close_dates.rename(columns={'日期': '上市日期'}, inplace=True)
    
    # 合并到原数据以获取每个公司的上市日期
    df = df.merge(first_close_dates, on='公司简称', how='left')
    
    # 收集上市时的控股股东和实际控制人
    subject_holders = set()
    for company in df['公司简称'].unique():
        ipo_date = first_close_dates.loc[first_close_dates['公司简称'] == company, '上市日期'].iloc[0]
        # 筛选上市日期的控股股东和实际控制人
        mask = (df['公司简称'] == company) & (df['日期'] == ipo_date) & (df['股东身份'].isin(['控股股东', '实际控制人']))
        holders = df[mask]['股东'].unique()
        subject_holders.update([(company, h) for h in holders])
    
    # 标记subject列
    df['cu_12_1_subject'] = df.apply(lambda row: (row['公司简称'], row['股东']) in subject_holders, axis=1)
    
    # 2. 验证触发条件：首次披露减持计划且方式正确
    # 找到每个股东首次披露减持计划的日期
    first_disclosures = df[df['存在减持计划']].groupby(['公司简称', '股东'])['计划披露日'].min().reset_index()
    first_disclosures.rename(columns={'计划披露日': '首次计划披露日'}, inplace=True)
    
    # 合并到原数据
    df = df.merge(first_disclosures, on=['公司简称', '股东'], how='left')
    
    # 条件判断
    valid_condition = (
        df['存在减持计划'] &
        (df['计划披露日'] == df['首次计划披露日']) &
        df['减持方式'].isin(['竞价交易', '大宗交易'])
    )
    df['cu_12_1_condition'] = valid_condition
    
    # 3. 验证约束条件：最近20个交易日无收盘价低于发行价（复权后）
    # 计算复权收盘价
    df['复权收盘价'] = df['收盘价'] * df['复权因子']
    df['收盘价低于发行价'] = df['复权收盘价'] < df['发行价格']
    
    # 按公司分组，按日期排序，计算每个日期前20个交易日的窗口
    df_sorted = df.sort_values(['公司简称', '日期'])
    df_sorted['最近20日存在收盘价低于发行价'] = df_sorted.groupby('公司简称')['收盘价低于发行价'].transform(
        lambda x: x.rolling(20, min_periods=1).apply(lambda y: y.any(), raw=True)
    )
    
    # 创建辅助字典
    aux_dict = df_sorted.set_index(['公司简称', '日期'])['最近20日存在收盘价低于发行价'].to_dict()
    
    # 处理符合条件的行
    mask = df['cu_12_1_subject'] & df['cu_12_1_condition']
    # 获取计划披露日对应的标记
    df.loc[mask, 'cu_12_1_constraint'] = df[mask].apply(
        lambda row: aux_dict.get((row['公司简称'], row['计划披露日']), True), axis=1
    )
    # 取反，因为约束是不得存在该情况
    df.loc[mask, 'cu_12_1_constraint'] = ~df.loc[mask, 'cu_12_1_constraint']
    
    # 清理中间列
    df.drop(columns=['上市日期', '首次计划��露日', '复权收盘价', '收盘价低于发行价'], inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_cu_12_1(df)
df

---

## cu_12_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 | 一致行动人 |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constrain | 不得存在下列情形：最近20个交易日内，上市公司任一交易日股票收盘价低于最近一个会计年度或者最近一期财务会计报告期末每股归属于上市公司股东的净资产 |
| contextual_info | 股票收盘价以最近一个会计年度或者最近一期财务会计报告资产负债表日为基准分别向后复权计算 |
| note | 不考虑已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况; 公司上市的头30个交易日不会披露减持计划; 不考虑一致行动人; "最近一个会计年度或者最近一期财务会计报告期末"理解为公司当前最新的财务数据即可, 不用分别形成两个复权收盘价再分别比较; 股票收盘价以最近一个会计年度或者最近一期财务会计报告资产负债表日为基准分别向后复权计算, 如果代码正确而取到此日的收盘价为空说明当日未上市, 不用特殊处理, 保留其为空即可 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2204 |
| completion_tokens | 7932 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_12_2(df):
    '''
    检查cu_12_2的合规性：
    主体：控股股东 | 实际控制人
    条件：通过竞价/大宗交易首次披露减持计划且不在上市头30日
    约束：前20个交易日收盘价均不低于最新每股净资产
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_12_2_subject'] = False
    df['cu_12_2_condition'] = False
    df['cu_12_2_constraint'] = None
    
    # 1. 验证责任主体
    df['cu_12_2_subject'] = df['股东身份'].isin(['控股股东', '实际控制人'])
    
    # 2. 验证触发条件
    # 计算交易日序号（上市后第N个交易日）
    df['交易日序号'] = df.groupby('公司简称')['日期'].rank(method='dense', ascending=True).astype(int)
    
    # 条件判断要素
    valid_method = df['减持方式'].isin(['竞价交易', '大宗交易'])
    has_plan = df['存在减持计划']
    not_new = df['交易日序号'] > 30  # 排除上市头30日
    
    # 计算首次披露标记
    plan_mask = df['存在减持计划']
    df['计划披露日'] = df['计划披露日'].where(plan_mask)  # 仅保留真实披露日
    df['首次计划披露日'] = df[plan_mask].groupby(
        ['公司简称', '股东'])['计划披露日'].transform('min')
    
    is_first = (df['计划披露日'] == df['首次计划披露日']) & has_plan
    
    # 综合条件判断
    df['cu_12_2_condition'] = valid_method & has_plan & not_new & is_first
    
    # 3. 验证约束条件（仅对符合主体和条件的记录进行检查）
    # 生成公司维度的交易日序列
    company_daily = df.groupby(['公司简称', '日期']).agg({
        '收盘价': 'first',
        '复权因子': 'first',
        '每股净资产': 'first'
    }).reset_index().sort_values(['公司简称', '日期'])
    
    # 计算复权价格并标记低于NAV的情况
    company_daily['复权收盘价'] = company_daily['收盘价'] * company_daily['复权因子']
    company_daily['低于NAV'] = company_daily['复权收盘价'] < company_daily['每股净资产']
    
    # 计算前20个交易日内是否存在违规
    company_daily['前20日违规'] = company_daily.groupby('公司简称')['低于NAV'].transform(
        lambda x: x.rolling(20, min_periods=1).max().shift(1)
    )
    
    # 合并违规标记到原始数据
    df = df.merge(
        company_daily[['公司简称', '日期', '前20日违规']],
        on=['公司简称', '日期'],
        how='left'
    )
    
    # 设置约束条件标记
    mask = df['cu_12_2_subject'] & df['cu_12_2_condition']
    df.loc[mask, 'cu_12_2_constraint'] = ~df.loc[mask, '前20日违规'].fillna(False)
    
    # 清理中间列
    df.drop(['交易日序号', '首次计划披露日', '前20日违规'], axis=1, errors='ignore', inplace=True)
    
    return df

In [ ]:
df = check_cu_12_2(df)
df

---

## cu_12_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司控股股东 | 实际控制人 | 一致行动人 |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constrain | 不得存在下列情形：上市公司最近一期经审计的财务报告的归属于上市公司股东的净利润为负 |
| contextual_info | nan |
| note | 不考虑中国证监会另有规定的情况; 不考虑一致行动人; 所有财务数据均已经过审计 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2073 |
| completion_tokens | 5498 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_12_3(df):
    '''
    检查cu_12_3的合规性：
    主体：控股股东或实际控制人
    条件：首次通过竞价或大宗交易减持计划
    约束：公司最近一期净利润非负
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_12_3_subject'] = False
    df['cu_12_3_condition'] = False
    df['cu_12_3_constraint'] = None
    
    # 1. 验证责任主体
    df['cu_12_3_subject'] = df['股东身份'].isin(['控股股东', '实际控制人'])
    
    # 2. 验证触发条件
    # 计算每个股东首次披露日（仅考虑有效减持计划）
    if df['存在减持计划'].any():
        # 生成股东维度首次披露日映射
        first_disclosure = (
            df[df['存在减持计划']]
            .groupby(['公司简称', '股东'])['计划披露日']
            .min()
            .reset_index()
            .rename(columns={'计划披露日': '首次披露日'})
        )
        
        # 合并首次披露日信息
        df = df.merge(
            first_disclosure,
            on=['公司简称', '股东'],
            how='left'
        )
        
        # 判断条件有效性
        valid_condition = (
            (df['计划披露日'] == df['首次披露日']) &
            df['减持方式'].isin(['竞价交易', '大宗交易']) &
            df['存在减持计划']
        )
        df['cu_12_3_condition'] = valid_condition
        df.drop(columns=['首次披露日'], inplace=True)
    else:
        df['cu_12_3_condition'] = False
    
    # 3. 验证约束条件（独立标记）
    df['cu_12_3_constraint'] = df['净利润'] >= 0
    
    # 处理空值（根据业务逻辑填充）
    df['cu_12_3_constraint'].fillna(False, inplace=True)
    
    return df

In [ ]:
df = check_cu_12_3(df)
df

---

## cu_12_4


| 字段 | 内容 |
|------|------|
| subject | 上市时控股股东 | 上市时实际控制人（上市后不再具备相关主体身份的也应当遵守） |
| condition | 计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constrain | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 |
| contextual_info | 股票收盘价以公开发行股票并上市之日为基准向后复权计算 |
| note | 不考虑中国证监会另有规定的情况 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2088 |
| completion_tokens | 10667 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_12_4(df):
    '''
    检查cu_12_4的合规性：
    主体：上市时控股股东 | 上市时实际控制人（即使之后不再具备身份）
    条件：计划通过竞价/大宗减持且首次披露
    约束：披露前20个交易日内收盘价（复权）均不低于发行价
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_12_4_subject'] = False
    df['cu_12_4_condition'] = False
    df['cu_12_4_constraint'] = None

    # 1. 验证责任主体：上市时的控股股东或实际控制人
    # 确定每个公司的上市日期（第一个收盘价非空的日期）
    ipo_dates = df[df['收盘价'].notna()].groupby('公司简称')['日期'].min()

    # 收集每个公司在上市日的控股股东和实际控制人
    subject_holders = {}
    for company, ipo_date in ipo_dates.items():
        ipo_day = df[(df['公司简称'] == company) & (df['日期'] == ipo_date)]
        holders = ipo_day[ipo_day['股东身份'].isin(['控股股东', '实际控制人'])]['股东'].unique()
        subject_holders[company] = set(holders)

    # 标记subject
    df['cu_12_4_subject'] = df.apply(
        lambda row: row['股东'] in subject_holders.get(row['公司简称'], set()),
        axis=1
    )

    # 2. 验证触发条件：减持方式为竞价/大宗，存在减持计划，且首次披露
    # 条件验证
    valid_method = df['减持方式'].isin(['竞价交易', '大宗交易'])
    has_plan = df['存在减持计划']
    
    # 计算首次披露标记
    mask_plan = df['存在减持计划']
    min_dates = df[mask_plan].groupby(['公司简称', '股东'])['计划披露日'].transform('min')
    df['is_first_disclosure'] = False
    df.loc[mask_plan, 'is_first_disclosure'] = df.loc[mask_plan, '计划披露日'] == min_dates

    # 组合条件
    valid_condition = valid_method & has_plan & df['is_first_disclosure']
    df['cu_12_4_condition'] = valid_condition

    # 3. 验证约束条件：最近20个交易日复权收盘价均不低于发行价
    # 预处理复权数据
    df['复权收盘价'] = df['收盘价'] * df['复权因子']
    company_data = {}
    for company, group in df.groupby('公司简称'):
        sorted_group = group.sort_values('日期')
        company_data[company] = {
            'dates': sorted_group['日期'].tolist(),
            'adjusted_close': sorted_group['复权收盘价'].tolist(),
            'ipo_price': sorted_group['发行价格'].iloc[0]
        }

    # 定义约束检查函数
    def check_price_constraint(row):
        if not (row['cu_12_4_subject'] and row['cu_12_4_condition']):
            return None
        company = row['公司简称']
        disclosure_date = row['计划披露日']
        data = company_data.get(company)
        if not data:
            return False
        
        try:
            idx = data['dates'].index(disclosure_date)
        except ValueError:
            return False
        
        start_idx = max(0, idx - 19)
        window_prices = data['adjusted_close'][start_idx:idx+1]
        return all(price >= data['ipo_price'] for price in window_prices)

    # 应用约束检查
    mask_constraint = df['cu_12_4_subject'] & df['cu_12_4_condition']
    df.loc[mask_constraint, 'cu_12_4_constraint'] = df[mask_constraint].apply(check_price_constraint, axis=1)

    # 清理临时列
    df.drop(columns=['is_first_disclosure', '复权收盘价'], inplace=True, errors='ignore')

    return df

In [ ]:
df = check_cu_12_4(df)
df

---

## cu_12_5


| 字段 | 内容 |
|------|------|
| subject | 公开发行时持股5%以上的第一大股东 | 一致行动人 |
| condition | 上市公司在公开发行股票并上市时披露为无控股股东、实际控制人, 且计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的 |
| constrain | 不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格 |
| contextual_info | 股票收盘价以公开发行股票并上市之日为基准向后复权计算 |
| note | 不考虑中国证监会另有规定的情况; 不考虑一致行动人; 假设数据库固定"公司简称"后, "股东"列包含该公司的全部股东的记录 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2126 |
| completion_tokens | 11613 |


### 代码实现

In [ ]:
import pandas as pd
import numpy as np

def check_cu_12_5(df):
    '''
    检查cu_12_5的合规性：
    主体：公开发行时持股5%以上的第一大股东
    条件：公司无控股股东/实际控制人，通过竞价/大宗减持，首次披露减持计划
    约束：最近20个交易日内收盘价不低于发行价（复权后）
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_12_5_subject'] = False
    df['cu_12_5_condition'] = False
    df['cu_12_5_constraint'] = None

    # 1. 处理责任主体：公开发行时持股5%以上的第一大股东
    # 确定各公司上市日期（首个收盘价非空日期）
    ipo_dates = df[df['收盘价'].notna()].groupby('公司简称')['日期'].min().reset_index()
    ipo_dates.columns = ['公司简称', '上市日期']
    df = df.merge(ipo_dates, on='公司简称', how='left')

    # 获取上市当日的股东数据
    ipo_shareholders = df[df['日期'] == df['上市日期']][['公司简称', '股东', '持股比例']]
    # 计算各公司最大持股比例
    max_holdings = ipo_shareholders.groupby('公司简称')['持股比例'].max().reset_index()
    max_holdings.columns = ['公司简称', 'max_holding']
    ipo_shareholders = ipo_shareholders.merge(max_holdings, on='公司简称')
    # 筛选符合条件的股东（持股比例等于最大值且≥5%）
    valid_subjects = ipo_shareholders[
        (ipo_shareholders['持股比例'] == ipo_shareholders['max_holding']) &
        (ipo_shareholders['持股比例'] >= 0.05)
    ][['公司简称', '股东']]
    # 创建有效主体集合
    valid_pairs = set(zip(valid_subjects['公司简称'], valid_subjects['股东']))
    # 标记责任主体
    df['cu_12_5_subject'] = df.apply(
        lambda x: (x['公司简称'], x['股东']) in valid_pairs, axis=1
    )

    # 2. 处理触发条件
    # 检查公司是否无控股股东/实际控制人（上市当日）
    control_check = df[df['日期'] == df['上市日期']].groupby('公司简称').apply(
        lambda g: g['股东身份'].isin(['控股股东', '实际控制人']).any()
    ).reset_index()
    control_check.columns = ['公司简称', 'has_control']
    df = df.merge(control_check, on='公司简称', how='left')

    # 确定首次披露减持计划的日期
    first_disclosures = df[df['存在减持计划']].groupby(['公司简称', '股东'])['日期'].min().reset_index()
    first_disclosures.columns = ['公司简称', '股东', 'first_disclosure']
    df = df.merge(first_disclosures, on=['公司简称', '股东'], how='left')

    # 复合条件判断
    condition_mask = (
        (~df['has_control']) &  # 无控股股东/实际控制人
        df['减持方式'].isin(['竞价交易', '大宗交易']) &  # 减持方式符合
        df['存在减持计划'] &  # 存在减持计划
        (df['日期'] == df['first_disclosure'])  # 是首次披露
    )
    df.loc[condition_mask, 'cu_12_5_condition'] = True

    # 3. 处理约束条件
    # 计算复权收盘价
    df['复权收盘价'] = df['收盘价'] * df['复权因子']
    
    # 按公司分组计算滚动20日最低价
    df_sorted = df.sort_values(['公司简称', '日期'])
    df_sorted['window_min'] = df_sorted.groupby('公司简称')['复权收盘价'].rolling(
        window=20, min_periods=1
    ).min().reset_index(level=0, drop=True)
    
    # 合并回原始df
    df = df.merge(
        df_sorted[['window_min']],
        left_index=True,
        right_index=True,
        how='left'
    )
    
    # 仅对符合条件的数据进行约束检查
    constraint_mask = df['cu_12_5_condition']
    df.loc[constraint_mask, 'cu_12_5_constraint'] = (
        df.loc[constraint_mask, 'window_min'] >= df.loc[constraint_mask, '发行价格']
    )

    # 清理中间列
    df.drop(
        columns=['上市日期', 'has_control', 'first_disclosure', '复权收盘价', 'window_min'],
        inplace=True,
        errors='ignore'
    )

    return df

In [ ]:
df = check_cu_12_5(df)
df

---

# Law Article 13

## cu_13_2


| 字段 | 内容 |
|------|------|
| subject | 大股东 |
| condition | 通过协议转让方式减持股份导致出让方不再具有大股东身份, 且处在减持后6个月内 |
| constrain | 继续遵守本指引第四条规定 |
| contextual_info | nan |
| note | subject项默认为True并且不再处理; 6个月按180个自然日计算, 含处罚日当日; constraint项目不处理, 留为None;  |
| relation | should_include |
| target | Law_4 |
| type | 数值化执行单元 |
| comments | 涉及到身份转换的条目, 需要人为介入. 此条目的subject应默认正确, 因为其已经蕴含在condition之中.  |
| prompt_tokens | 2049 |
| completion_tokens | 6101 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_13_2(df):
    '''
    检查cu_13_2的合规性：
    主体：大股东（默认为True）
    条件：通过协议转让减持导致不再是大股东，且在减持后6个月内
    约束：不处理
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_13_2_subject'] = True  # 根据要求，subject默认为True
    df['cu_13_2_condition'] = False
    df['cu_13_2_constraint'] = None  # 根据要求，constraint留为None
    
    # 筛选触发事件条件
    is_protocol_transfer = df['减持方式'] == '协议转让'
    is_no_longer_major = ~(
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) | 
        (df['持股比例'] >= 0.05)
    )
    trigger_mask = is_protocol_transfer & is_no_longer_major
    
    # 提取有效触发事件
    if trigger_mask.any():
        triggers = df.loc[trigger_mask, ['公司简称', '股东', '日期']].copy()
        triggers['end_date'] = triggers['日期'] + pd.Timedelta(days=180)
        
        # 构建时间范围判断器
        def check_date_range(row):
            mask = (
                (triggers['公司简称'] == row['公司简称']) &
                (triggers['股东'] == row['股东']) &
                (row['日期'] >= triggers['日期']) &
                (row['日期'] <= triggers['end_date'])
            )
            return mask.any()
        
        # 应用日期范围检查
        df['cu_13_2_condition'] = df.apply(check_date_range, axis=1)
    
    return df

In [ ]:
df = check_cu_13_2(df)
df

---

## cu_13_3


| 字段 | 内容 |
|------|------|
| subject | 控股股东 | 实际控制人 |
| condition | 通过协议转让方式减持股份导致其不再具有控股股东、实际控制人身份, 且处于减持后的6个月内 |
| constrain | 应当继续遵守本指引第十二条第一款第二、三项规定 |
| contextual_info | nan |
| note | subject项默认为True并且不再处理; 6个月按180个自然日计算, 含处罚日当日; constraint项目不处理, 留为None;  |
| relation | should_include |
| target | MEU_12_1;MEU_12_2 |
| type | 数值化执行单元 |
| comments | 涉及到身份转换的条目, 需要人为介入. 此条目的subject应默认正确, 因为其已经蕴含在condition之中.  |
| prompt_tokens | 2063 |
| completion_tokens | 6886 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_13_3(df):
    '''
    检查cu_13_3的合规性：
    主体：控股股东 | 实际控制人（标记为True）
    条件：通过协议转让减持导致失去身份且处于减持后6个月内
    约束：不处理
    '''
    df = df.copy()
    
    # 1. 标记责任主体（默认全部True）
    df['cu_13_3_subject'] = True
    
    # 2. 初始化条件标记列
    df['cu_13_3_condition'] = False
    df['cu_13_3_constraint'] = None  # 约束不处理
    
    # 3. 按公司和股东分组处理
    for (company, shareholder), group in df.groupby(['公司简称', '股东']):
        group_sorted = group.sort_values('日期')
        
        # 遍历寻找符合条件的减持事件
        for idx in group_sorted.index:
            current_row = group_sorted.loc[idx]
            
            # 条件检查：协议转让减持且当前具有主体身份
            if (current_row['减持方式'] == '协议转让' 
                and current_row['当日减持比例'] > 0
                and current_row['股东身份'] in ['控股股东', '实际控制人']):
                
                # 检查后续身份是否发生变化
                subsequent_status = group_sorted.loc[idx:]['股东身份']
                if (~subsequent_status.isin(['控股股东', '实际控制人'])).any():
                    # 计算时间窗口
                    event_date = current_row['日期']
                    end_date = event_date + pd.Timedelta(days=180)
                    
                    # 标记时间窗口内的行
                    mask = (
                        (df['公司简称'] == company) 
                        & (df['股东'] == shareholder)
                        & (df['日期'] >= event_date)
                        & (df['日期'] <= end_date)
                    )
                    df.loc[mask, 'cu_13_3_condition'] = True

    return df

In [ ]:
df = check_cu_13_3(df)
df

---

# Law Article 14

## cu_14_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司因涉嫌证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2016 |
| completion_tokens | 6851 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_14_1(df):
    '''
    检查cu_14_1的合规性：
    主体：上市公司董监高
    条件：公司因涉嫌证券期货违法犯罪被立案调查/侦查
    约束：不得减持股份
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_14_1_subject'] = False
    df['cu_14_1_condition'] = False
    df['cu_14_1_constraint'] = None

    # 1. 验证责任主体
    df['cu_14_1_subject'] = (df['股东身份'] == '董监高')

    # 2. 构建公司调查状态时间线
    def build_investigation_status(df):
        # 提取公司维度事件并去重
        company_events = df[['公司简称', '日期', '公司涉嫌证券期货违法犯罪事件']]\
            .drop_duplicates(subset=['公司简称', '日期'])\
            .sort_values(['公司简称', '日期'])

        # 按公司分组处理事件序列
        def process_company(group):
            group = group.sort_values('日期')
            in_investigation = False
            status = []
            for _, row in group.iterrows():
                event = row['公司涉嫌证券期货违法犯罪事件']
                if event in ['被中国证监会及其派出机构立案调查', '被司法机关立案侦查']:
                    in_investigation = True
                elif event in ['中国证监会及其派出机构立案调查结束', '司法机关立案侦查结束']:
                    in_investigation = False
                status.append(in_investigation)
            return pd.DataFrame({
                '日期': group['日期'],
                'in_investigation': status
            })
        
        # 生成全量公司状态表
        status_df = company_events.groupby('公司简称', group_keys=False).apply(process_company)
        return status_df.reset_index(drop=True)

    # 合并调查状态到原始数据
    status_df = build_investigation_status(df)
    df = df.merge(status_df, on=['公司简称', '日期'], how='left')
    df['cu_14_1_condition'] = df['in_investigation'].fillna(False)

    # 3. 验证减持约束（仅对复合主体+条件的记录进行检查）
    mask = df['cu_14_1_subject'] & df['cu_14_1_condition']
    df.loc[mask, 'cu_14_1_constraint'] = (df.loc[mask, '当日减持比例'] <= 0)

    # 清理中间列
    df.drop(columns=['in_investigation'], inplace=True, errors='ignore')

    return df

In [ ]:
df = check_cu_14_1(df)
df

---

## cu_14_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司因涉嫌证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2027 |
| completion_tokens | 7149 |


### 代码实现

In [ ]:
import pandas as pd
from pandas.tseries.offsets import DateOffset

def check_cu_14_2(df):
    '''
    检查cu_14_2的合规性：
    主体：上市公司董监高
    条件：公司因涉嫌证券期货违法犯罪被立案调查/侦查，且在处罚/判决后未满6个月
    约束：不得减持股份
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_14_2_subject'] = False
    df['cu_14_2_condition'] = False
    df['cu_14_2_constraint'] = None
    
    # 1. 验证责任主体
    df['cu_14_2_subject'] = (df['股东身份'] == '董监高')
    
    # 2. 验证触发条件
    # 获取公司调查和处罚事件时间轴
    investigation_mask = df['公司涉嫌证券期货违法犯罪事件'].isin(['被中国证监会及其派出机构立案调查', '被司法机关立案侦查'])
    punishment_mask = df['公司涉嫌证券期货违法犯罪事件'].isin(['行政处罚决定作出', '刑事判决作出'])
    
    # 构建公司时间线字典
    company_timelines = {}
    for company, group in df.groupby('公司简称'):
        # 收集调查事件时间
        investigations = group[investigation_mask]['日期'].tolist()
        # 收集有效处罚时间（需有前置调查）
        punishments = [
            p_date for p_date in group[punishment_mask]['日期']
            if any(i_date < p_date for i_date in investigations)
        ]
        company_timelines[company] = {
            'valid_punishments': punishments,
            'investigation_dates': investigations
        }
    
    # 条件检查函数
    def check_condition(row):
        company = row['公司简称']
        current_date = row['日期']
        punishments = company_timelines.get(company, {}).get('valid_punishments', [])
        
        for p_date in punishments:
            if (current_date >= p_date) and (current_date <= p_date + DateOffset(months=6)):
                return True
        return False
    
    df['cu_14_2_condition'] = df.apply(check_condition, axis=1)
    
    # 3. 验证约束条件
    constraint_mask = df['cu_14_2_subject'] & df['cu_14_2_condition']
    df.loc[constraint_mask, 'cu_14_2_constraint'] = ~(df.loc[constraint_mask, '当日减持比例'] > 0)
    
    return df

In [ ]:
df = check_cu_14_2(df)
df

---

## cu_14_3


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉嫌与该上市公司有关的证券期货违法犯罪，在被中国证监会及其派出机构立案调查或者被司法机关立案侦查期间 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2019 |
| completion_tokens | 5528 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_14_3(df):
    '''
    检查cu_14_3的合规性：
    主体：上市公司董监高
    条件：因涉嫌证券期货违法犯罪被立案调查/侦查
    约束：不得减持股份
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_14_3_subject'] = False
    df['cu_14_3_condition'] = False
    df['cu_14_3_constraint'] = True  # 默认视为合规

    # 1. 验证责任主体：上市公司董监高
    df['cu_14_3_subject'] = df['股东身份'] == '董监高'

    # 2. 验证触发条件：处于立案调查/侦查期间
    investigation_status = [
        '被中国证监会及其派出机构立案调查',
        '被司法机关立案侦查'
    ]
    df['cu_14_3_condition'] = df['股东涉嫌证券期货违法犯罪事件'].isin(investigation_status)

    # 3. 验证约束条件：不得减持（仅对符合主体+条件的行进行约束检查）
    mask = df['cu_14_3_subject'] & df['cu_14_3_condition']
    df.loc[mask, 'cu_14_3_constraint'] = (df.loc[mask, '当日减持比例'] <= 0)

    return df

In [ ]:
df = check_cu_14_3(df)
df

---

## cu_14_4


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉嫌与该上市公司有关的证券期货违法犯罪被中国证监会及其派出机构立案调查或者被司法机关立案侦查，在行政处罚决定、刑事判决作出之后未满6个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | 6个月按180个自然日计算, 含处罚日当日 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2044 |
| completion_tokens | 9808 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_14_4(df):
    '''
    检查cu_14_4的合规性：
    主体：上市公司董监高
    条件：因涉嫌证券期货违法犯罪被立案调查/侦查，且在处罚后未满180天
    约束：不得减持股份
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_14_4_subject'] = False
    df['cu_14_4_condition'] = False
    df['cu_14_4_constraint'] = None
    
    # 1. 验证责任主体：股东身份是董监高
    df['cu_14_4_subject'] = df['股东身份'] == '董监高'
    
    # 2. 验证触发条件
    # 判断是否被立案调查或侦查过
    df['has_been_investigated'] = df.groupby(['公司简称', '股东'])['股东涉嫌证券期货违法犯罪事件'].transform(
        lambda x: x.isin(['被中国证监会及其派出机构立案调查', '被司法机关立案侦查']).any()
    )
    
    # 获取每个分组的最新处罚日期
    df['latest_penalty_date'] = df.groupby(['公司简称', '股东'])['日期'].transform(
        lambda x: x.where(
            df['股东涉嫌证券期货违法犯罪事件'].isin(['行政处罚决定作出', '刑事判决作出'])
        ).max()
    )
    
    # 计算自然日差
    df['days_since_penalty'] = (df['日期'] - df['latest_penalty_date']).dt.days
    
    # 条件判断：被立案过，且当前日期在处罚后的180天内
    df['cu_14_4_condition'] = (
        df['has_been_investigated'] & 
        (df['days_since_penalty'] >= 0) & 
        (df['days_since_penalty'] <= 180)
    )
    
    # 3. 验证约束条件：独立检查减持行为
    df['cu_14_4_constraint'] = (df['当日减持比例'] <= 0)
    
    # 清理中间列
    df.drop(columns=['has_been_investigated', 'latest_penalty_date', 'days_since_penalty'], 
            inplace=True, errors='ignore')
    
    return df

In [ ]:
df = check_cu_14_4(df)
df

---

## cu_14_5


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉及证券期货违法，被中国证监会行政处罚，罚没款尚未足额缴纳，且不存在法律、行政法规另有规定或减持资金用于缴纳罚没款的情况 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | 不考虑法律、行政法规另有规定或减持资金用于缴纳罚没款的情况 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | 暂不处理, 有空再在模拟数据上加上 |
| prompt_tokens | 2046 |
| completion_tokens | 3791 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_14_5(df):
    '''
    检查cu_14_5的合规性：
    主体：上市公司董监高
    条件：因证券期货违法被证监会行政处罚且罚没款未缴
    约束：不得减持股份
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_14_5_subject'] = False
    df['cu_14_5_condition'] = False
    df['cu_14_5_constraint'] = None

    # 1. 验证责任主体
    df['cu_14_5_subject'] = df['股东身份'] == '董监高'

    # 2. 验证触发条件
    # 获取行政处罚日期
    penalty_events = df[df['股东涉嫌证券期货违法犯罪事件'] == '行政处罚决定作出']
    if not penalty_events.empty:
        penalty_dates = penalty_events.groupby(['公司简称', '股东'])['日期'].min().reset_index()
        penalty_dates.rename(columns={'日期':'penalty_date'}, inplace=True)
        df = df.merge(penalty_dates, on=['公司简称', '股东'], how='left')
        
        # 条件成立判断
        condition_mask = (
            df['cu_14_5_subject'] & 
            df['penalty_date'].notna() & 
            (df['日期'] >= df['penalty_date'])
        )
        df.loc[condition_mask, 'cu_14_5_condition'] = True
        df.drop(columns=['penalty_date'], inplace=True)
    else:
        df['cu_14_5_condition'] = False

    # 3. 验证约束条件
    constraint_mask = df['cu_14_5_subject'] & df['cu_14_5_condition']
    df.loc[constraint_mask, 'cu_14_5_constraint'] = (df.loc[constraint_mask, '当日减持比例'] <= 0)

    return df

In [ ]:
df = check_cu_14_5(df)
df

---

## cu_14_6


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 本人因涉及与本上市公司有关的违法违规，被证券交易所公开谴责未满三个月 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | 三个月按照90个自然日计算, 含处罚日当日;  |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2024 |
| completion_tokens | 6691 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_14_6(df):
    '''
    检查cu_14_6的合规性：
    主体：上市公司董监高
    条件：被本所公开谴责未满90自然日
    约束：不得减持股份
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_14_6_subject'] = False
    df['cu_14_6_condition'] = False
    df['cu_14_6_constraint'] = None

    # 1. 验证责任主体：上市公司董监高
    df['cu_14_6_subject'] = (df['股东身份'] == '董监高')

    # 2. 验证触发条件：被公开谴责未满90自然日
    # 生成谴责日期映射表
    condemn_df = df[df['股东涉嫌证券期货违法犯罪事件'] == '被本所公开谴责']
    if not condemn_df.empty:
        # 按股东分组获取所有谴责日期
        condemn_dates = (
            condemn_df.groupby(['公司简称', '股东'])['日期']
            .apply(lambda x: x.tolist())
            .reset_index(name='condemn_dates')
        )
        
        # 合并谴责日期到主表
        df = df.merge(condemn_dates, on=['公司简称', '股东'], how='left')
        
        # 定义日期检查函数
        def check_condemn_window(row):
            if not isinstance(row.get('condemn_dates'), list):
                return False
            current_date = row['日期']
            for d in row['condemn_dates']:
                if d <= current_date <= d + pd.Timedelta(days=89):
                    return True
            return False
        
        df['cu_14_6_condition'] = df.apply(check_condemn_window, axis=1)
        df.drop(columns=['condemn_dates'], inplace=True)
    else:
        df['cu_14_6_condition'] = False

    # 3. 验证约束条件：不得减持
    mask = df['cu_14_6_subject'] & df['cu_14_6_condition']
    df.loc[mask, 'cu_14_6_constraint'] = (df.loc[mask, '当日减持比例'] <= 0)

    return df

In [ ]:
df = check_cu_14_6(df)
df

---

## cu_14_7


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司股票因可能触及重大违法强制退市情形而被本所实施退市风险警示，且处于本所规定的限制转让的期限内 |
| constrain | 不得减持其所持有的本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2024 |
| completion_tokens | 8251 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_14_7(df):
    '''
    检查cu_14_7的合规性：
    主体：上市公司董监高
    条件：公司因重大违法被实施退市风险警示且处于限制转让期
    约束：不得减持股份
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_14_7_subject'] = False
    df['cu_14_7_condition'] = False
    df['cu_14_7_constraint'] = None
    
    # 1. 验证责任主体
    df['cu_14_7_subject'] = df['股东身份'] == '董监高'
    
    # 2. 预处理公司限制转让期
    def get_restriction_periods(group):
        periods = []
        current_start = None
        sorted_group = group.sort_values('日期')
        
        for _, row in sorted_group.iterrows():
            event = row.get('公司涉嫌证券期货违法犯罪事件', None)
            if event == '被中国证监会及其派出机构立案调查' and current_start is None:
                current_start = row['日期']
            elif event in ['中国证监会及其派出机构立案调查结束', '行政处罚决定作出'] and current_start is not None:
                periods.append((current_start, row['日期']))
                current_start = None
        
        # 处理未闭环的立案调查
        if current_start is not None:
            periods.append((current_start, group['日期'].max()))
        
        return periods
    
    # 获取各公司限制期列表
    company_periods = df.groupby('公司简称', group_keys=False).apply(
        lambda g: pd.Series([get_restriction_periods(g)], name='periods')
    ).reset_index()
    
    # 3. 标记条件有效性
    def check_condition(row, company_periods_dict):
        periods = company_periods_dict.get(row['公司简称'], [])
        for (start, end) in periods:
            if start <= row['日期'] <= end:
                return True
        return False
    
    company_periods_dict = company_periods.set_index('公司简称')['periods'].to_dict()
    df['cu_14_7_condition'] = df.apply(
        lambda row: check_condition(row, company_periods_dict),
        axis=1
    )
    
    # 4. 验证约束条件（独立标记）
    df['cu_14_7_constraint'] = (df['当日减持比例'] <= 0)
    
    return df

In [ ]:
df = check_cu_14_7(df)
df

---

# Law Article 16

## cu_16_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司年度报告、半年度报告公告前15日内 |
| constrain | 不得买卖本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2004 |
| completion_tokens | 3027 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_16_1(df):
    '''
    检查cu_16_1的合规性：
    主体：上市公司董监高
    条件：处于定期报告公告前15日期间
    约束：禁止股份交易
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_16_1_subject'] = False
    df['cu_16_1_condition'] = False
    df['cu_16_1_constraint'] = None
    
    # 1. 验证责任主体
    df['cu_16_1_subject'] = df['股东身份'] == '董监高'
    
    # 2. 构建公司定期报告时间索引
    def build_report_dates(df):
        # 提取所有定期报告公告记录
        period_reports = df[df['公告类型'].isin(['年度报告', '半年度报告'])]
        # 按公司分组并去重
        return period_reports.groupby('公司简称')['公告日期'].apply(
            lambda x: x.dt.normalize().drop_duplicates().tolist()
        ).to_dict()
    
    company_report_dates = build_report_dates(df)
    
    # 3. 验证触发条件
    def check_condition(row):
        report_dates = company_report_dates.get(row['公司简称'], [])
        current_date = row['日期'].normalize()
        for rd in report_dates:
            if (rd - pd.Timedelta(days=15) <= current_date) and (current_date < rd):
                return True
        return False
    
    df['cu_16_1_condition'] = df.apply(check_condition, axis=1)
    
    # 4. 验证交易约束（基于减持行为判断）
    transaction_mask = df['cu_16_1_subject'] & df['cu_16_1_condition']
    df.loc[transaction_mask, 'cu_16_1_constraint'] = (
        df.loc[transaction_mask, '当日减持比例'] <= 0  # 无减持视为合规
    )
    
    # 处理可能存在的NaN值（当减持比例为NaN时视为无交易）
    df['cu_16_1_constraint'] = df['cu_16_1_constraint'].fillna(True)
    
    return df

In [ ]:
df = check_cu_16_1(df)
df

---

## cu_16_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 上市公司季度报告、业绩预告、业绩快报公告前5日内 |
| constrain | 不得买卖本公司股份 |
| contextual_info | nan |
| note | 只考虑季度报告情况 |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2012 |
| completion_tokens | 5836 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_16_2(df):
    '''
    检查cu_16_2的合规性：
    主体：上市公司董监高
    条件：处于季度报告公告前5个自然日内
    约束：不得买卖本公司股份
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_16_2_subject'] = False
    df['cu_16_2_condition'] = False
    df['cu_16_2_constraint'] = None

    # 1. 验证责任主体
    df['cu_16_2_subject'] = df['股东身份'] == '董监高'

    # 2. 验证触发条件
    # 获取季度报告公告信息
    quarterly_announcements = df[df['公告类型'] == '季度报告'][['公司简称', '公告日期']].drop_duplicates()
    
    if not quarterly_announcements.empty:
        # 计算有效区间
        quarterly_announcements['pre_start'] = quarterly_announcements['公告日期'] - pd.Timedelta(days=5)
        quarterly_announcements['pre_end'] = quarterly_announcements['公告日期'] - pd.Timedelta(days=1)
        
        # 合并有效区间到原始数据
        merged = df.merge(quarterly_announcements, on='公司简称', how='left')
        
        # 判断日期是否在有效区间内
        in_window = (merged['日期'] >= merged['pre_start']) & (merged['日期'] <= merged['pre_end'])
        condition_mask = in_window.groupby(merged.index).any()
        df['cu_16_2_condition'] = condition_mask

    # 3. 验证约束条件（所有行独立判断）
    df['cu_16_2_constraint'] = df['当日减持比例'] == 0

    return df

In [ ]:
df = check_cu_16_2(df)
df

---

# Law Article 17

## cu_17_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 | 离任六个月内的董监高 |
| condition | 每年通过集中竞价、大宗交易、协议转让等方式转让股份且不属于因司法强制执行、继承、遗赠、依法分割财产等导致股份变动 |
| constrain | 每年转让的股份不得超过其所持本公司股份总数的25% |
| contextual_info | nan |
| note | 不考虑因司法强制执行、继承、遗赠、依法分割财产等导致股份变动的情况; 6个月按180个自然日计算, 含处罚日当日; 年按照自然年计算; 计算持股数量限制时以持股比例为准, 而非绝对数量.  |
| relation | refer_to |
| target | MEU_18_1;MEU_18_2;MEU_18_3;MEU_18_4 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2173 |
| completion_tokens | 9368 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_17_1(df):
    '''
    检查cu_17_1的合规性：
    主体：上市公司董监高 | 离任六个月内的董监高
    条件：每年通过集中竞价、大宗交易、协议转让等方式转让股份且不属于例外情况
    约束：每年转让的股份不得超过所持本公司股份总数的25%
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_17_1_subject'] = False
    df['cu_17_1_condition'] = False
    df['cu_17_1_constraint'] = None
    
    # 1. 验证责任主体
    # 现任董监高
    is_current = df['股东身份'] == '董监高'
    # 离任董监高且离任日期在180天内
    is_former = (
        (df['股东身份'] == '离任董监高') &
        ((df['日期'] - df['离任日期']).dt.days <= 180)
    )
    df['cu_17_1_subject'] = is_current | is_former
    
    # 2. 验证触发条件
    valid_condition = (
        df['减持方式'].isin(['竞价交易', '大宗交易', '协议转让']) &
        (df['当日减持比例'] > 0)
    )
    df['cu_17_1_condition'] = valid_condition
    
    # 3. 验证约束条件
    # 添加年份列
    df['year'] = df['日期'].dt.year
    
    # 按公司、股东、年份分组获取年初持股比例
    df_sorted = df.sort_values(['公司简称', '股东', 'year', '日期'])
    year_start_shares = df_sorted.groupby(['公司简称', '股东', 'year'])['持股比例'].first().reset_index()
    year_start_shares.rename(columns={'持股比例': '年初持股比例'}, inplace=True)
    df = df.merge(year_start_shares, on=['公司简称', '股东', 'year'], how='left')
    
    # 计算累计减持比例
    df_sorted['累计减持比例'] = df_sorted.groupby(['公司简称', '股东', 'year'])['当日减持比例'].cumsum()
    df['累计减持比例'] = df_sorted['累计减持比例']
    
    # 判断约束条件
    df['cu_17_1_constraint'] = True
    mask = df['当日减持比例'] > 0
    df.loc[mask, 'cu_17_1_constraint'] = (
        df.loc[mask, '累计减持比例'] <= (df.loc[mask, '年初持股比例'] * 0.25)
    )
    
    # 处理空值（无减持记录时默认合规）
    df['cu_17_1_constraint'] = df['cu_17_1_constraint'].fillna(True)
    
    return df

In [ ]:
df = check_cu_17_1(df)
df

---

## cu_17_2


| 字段 | 内容 |
|------|------|
| subject | 上市公司董监高 |
| condition | 所持股份不超过1000股 |
| constrain | 可一次全部转让且不受前款转让比例限制 |
| contextual_info | nan |
| note | 不需要处理constrain项, 保留其为None |
| relation | exclude |
| target | MEU_17_1 |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 2017 |
| completion_tokens | 880 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_17_2(df):
    '''
    检查cu_17_2的合规性：
    主体：上市公司董监高
    条件：所持股份不超过1000股
    约束：保留为None
    '''
    df = df.copy()
    
    # 初始化标记列
    df['cu_17_2_subject'] = False
    df['cu_17_2_condition'] = False
    df['cu_17_2_constraint'] = None  # 根据要求保留为None
    
    # 1. 验证责任主体：上市公司现任董监高
    df.loc[df['股东身份'] == '董监高', 'cu_17_2_subject'] = True
    
    # 2. 验证触发条件：当日持股数量≤1000股
    # 根据数据说明，持股数量单位为股且数据完整
    df.loc[df['持股数量'] <= 1000, 'cu_17_2_condition'] = True
    
    return df

In [ ]:
df = check_cu_17_2(df)
df

---

# Law Article 23

## cu_23_1


| 字段 | 内容 |
|------|------|
| subject | 上市公司大股东 | 董监高 |
| condition | nan |
| constrain | 不得融券卖出本公司股份 |
| contextual_info | nan |
| note | nan |
| relation | nan |
| target | nan |
| type | 数值化执行单元 |
| comments | nan |
| prompt_tokens | 1999 |
| completion_tokens | 6797 |


### 代码实现

In [ ]:
import pandas as pd

def check_cu_23_1(df):
    '''
    检查cu_23_1的合规性：
    主体：上市公司大股东 | 董监高
    约束：禁止融券卖出本公司股份
    '''
    df = df.copy()

    # 初始化标记列
    df['cu_23_1_subject'] = False
    df['cu_23_1_condition'] = True  # 无触发条件
    df['cu_23_1_constraint'] = True  # 默认合规

    # 1. 验证责任主体
    # 大股东判断：身份标识或持股比例≥5%
    is_major = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    # 董监高判断
    is_director = df['股东身份'] == '董监高'
    df.loc[is_major | is_director, 'cu_23_1_subject'] = True

    # 3. 验证约束条件（仅对责任主体进行约束检查）
    # 融券卖出标记为违规
    mask = df['cu_23_1_subject'] & (df['减持方式'] == '融券卖出')
    df.loc[mask, 'cu_23_1_constraint'] = False

    return df

In [ ]:
df = check_cu_23_1(df)
df

---